In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import gc
import glob
import io
import IPython
import json
import numpy as np
import pathlib
import pandas as pd
import sys
import cv2
import math
from tqdm.notebook import tqdm
tqdm.pandas()

import matplotlib.pyplot as plt

from PIL import Image
import tensorflow as tf

INPUT_DIR = '/kaggle/input/tensorflow-great-barrier-reef/'
sys.path.insert(0, INPUT_DIR)
import greatbarrierreef

In [ ]:
print(tf.__version__)
print(tf.test.is_gpu_available())
print(tf.config.list_physical_devices('GPU'))

# Install TF Object Detection API & Download Pre-Trained Model

Created folders:
1. **api**: for storing tensorflow object detection model api files
2. **pre-trained-models**: for storing downloaded pretrained models including checkpoints and config
3. **my_models**: for storing trained model and new config
4. **data**: for storing tfrecords files and `label_map.pbtxt` file

## Create Configs and Folders 

In [ ]:
CUSTOM_MODEL_NAME = 'my_efficientdet_d2' 
PRETRAINED_MODEL_NAME = 'efficientdet_d2_coco17_tpu-32'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d2_coco17_tpu-32.tar.gz'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [ ]:
folders = {
    'APIMODEL_PATH': 'api',
    'DATA_PATH': 'data',
    'MODEL_PATH': 'my_models',
    'PRETRAINED_MODEL_PATH': 'pre-trained-models',
    'CHECKPOINT_PATH': os.path.join('my_models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('my_models',CUSTOM_MODEL_NAME, 'export')
}

files = {
    'PIPELINE_CONFIG':os.path.join(folders['MODEL_PATH'], CUSTOM_MODEL_NAME, 'pipeline.config'),
    'LABELMAP': os.path.join(folders['DATA_PATH'], LABEL_MAP_NAME),
    'VERIFICATION_SCRIPT': os.path.join(folders['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py'),
    'TRAINING_SCRIPT': os.path.join(folders['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py'),
    'EXPORTER_SCRIPT': os.path.join(folders['APIMODEL_PATH'], 'research', 'object_detection', 'exporter_main_v2.py')
}

In [ ]:
for path in folders.values():
    if not os.path.exists(path):
        !mkdir -p {path}

## Download Pre-Trained Model

In [ ]:
!wget {PRETRAINED_MODEL_URL} -P {folders['PRETRAINED_MODEL_PATH']}
!cd {folders['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

## Download TFODT API

In [ ]:
if not os.path.exists(os.path.join(folders['APIMODEL_PATH'], 'research', 'object_detection')):
    !git clone https://github.com/tensorflow/models {folders['APIMODEL_PATH']}

## Install TFODT API

In [ ]:
%%bash
cd api/research

protoc object_detection/protos/*.proto --python_out=.

# cp object_detection/packages/tf2/setup.py .
wget https://storage.googleapis.com/odml-dataset/others/setup.py
pip install -q --user .

pip install -q imagesize

In [ ]:
! python {files['VERIFICATION_SCRIPT']}

## Import TFODT API

In [ ]:
from object_detection.utils import dataset_util, label_map_util, config_util

from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format
import imagesize

# Prepare Data

## Split Dataset

In [ ]:
TRAINING_RATIO = 0.8
data_df = pd.read_csv(os.path.join(INPUT_DIR, 'train.csv'))

In [ ]:
split_index = int(TRAINING_RATIO * len(data_df))

while data_df.iloc[split_index - 1].sequence == data_df.iloc[split_index].sequence:
    split_index += 1

train_data_df = data_df.iloc[:split_index].sample(frac=1).reset_index(drop=True)
val_data_df = data_df.iloc[split_index:].sample(frac=1).reset_index(drop=True)

train_positive_count = len(train_data_df[train_data_df.annotations != '[]'])
val_positive_count = len(val_data_df[val_data_df.annotations != '[]'])

print('Training ratio (all samples):', f"{(float(len(train_data_df)) / (len(train_data_df) + len(val_data_df))):.2f}")
print('Training ratio (positive samples):', f"{(float(train_positive_count) / (train_positive_count + val_positive_count)):.2f}")

In [ ]:
train_data_df = train_data_df[train_data_df.annotations != '[]'].reset_index()
print('Number of positive images used for training:', len(train_data_df))
val_data_df = val_data_df[val_data_df.annotations != '[]'].reset_index()
print('Number of positive images used for validation:', len(val_data_df))

In [ ]:
train_data_df.head()

## Helper Functions

In [ ]:
def load_image_into_np(path):
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(io.BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np, boxes, classes, scores, category_index, unc=True):
    image_np_with_annotations = image_np.copy()
    viz_utils.visualize_boxes_and_labels_on_image_array(image_np_with_annotations,
                                                       boxes,
                                                       classes,
                                                       scores,
                                                       category_index,
                                                       use_normalized_coordinates=unc,
                                                       min_score_thresh=0.05)
    return image_np_with_annotations

def get_image_with_annotation(df, idx):
    row = df.iloc[idx]
    img = load_image_into_np(row.image_path)
    boxes = np.asarray(row.bboxes)
    num_boxes = len(boxes)
    classes = np.ones(num_boxes, dtype='int32')
    scores = np.ones(num_boxes)
    category_index = {1: {'id': 1, 'name': 'COTS'}}
    unc = True
    
    img = plot_detections(img, boxes, classes, scores, category_index, unc)
    return img


def get_bbox(row):
    bboxes = []
    annotations = json.loads(row.annotations.replace("'", '"'))
    for annotation in annotations:
            bboxes.append([annotation['y'] / row.height, 
                           annotation['x'] / row.width,
                           (annotation['y'] + annotation['height']) / row.height, 
                           (annotation['x'] + annotation['width']) / row.width])
    row["bboxes"] = bboxes
    return row

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row["image_path"])
    return row

def get_path(row):
    row["image_path"] = os.path.join(INPUT_DIR, "train_images", f'video_{row.video_id}', f'{row.video_frame}.jpg')
    return row

## Add Some Useful Columns

In [ ]:
train_data_df = train_data_df.progress_apply(get_path,axis=1)
val_data_df = val_data_df.progress_apply(get_path,axis=1)

train_data_df = train_data_df.progress_apply(get_imgsize,axis=1)
val_data_df = val_data_df.progress_apply(get_imgsize,axis=1)

train_data_df = train_data_df.progress_apply(get_bbox,axis=1)
val_data_df = val_data_df.progress_apply(get_bbox,axis=1)

In [ ]:
train_data_df.head()

## Visualize One Sample

In [ ]:
%matplotlib inline
idx = np.random.randint(0,train_data_df.shape[0]) 
img = get_image_with_annotation(train_data_df, idx)
plt.figure(figsize=(20,10))
plt.imshow(img)
plt.title(f"Image Index {idx}")
plt.show()

## Create TFRecord Files

In [ ]:
def create_tf_example(row):
    with tf.io.gfile.GFile(row["image_path"], 'rb') as fid:
        encoded_jpg = fid.read()

    height = row["height"]
    width = row["width"]
    filename = f'{row["video_id"]}:{row["video_frame"]}'.encode('utf8') 
    image_format = 'jpeg'.encode() 

    bb = row["bboxes"]
    
    xmins = [i[1] for i in bb]
    xmaxs = [i[3] for i in bb]     
    ymins = [i[0] for i in bb] 
    ymaxs = [i[2] for i in bb] 
            
    classes_text = ['COTS'.encode() for i in bb]
    classes_id = [1 for i in bb] 

    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes_id),
    }))
    
    return tf_example.SerializeToString()


def convert_to_tfrecord(data_df, filename):
    with tf.io.TFRecordWriter(os.path.join(folders["DATA_PATH"],filename)) as writer:
        for _, row in tqdm(data_df.iterrows()):
            tf_example = create_tf_example(row)
            writer.write(tf_example)

In [ ]:
convert_to_tfrecord(train_data_df, 'train.tfrec')
convert_to_tfrecord(val_data_df, 'valid.tfrec')

## Create Label Map

In [ ]:
labels = [{'name':'COTS', 'id':1}]

with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')
        
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

## Test TFRecords

In [ ]:
# def deserialize_example(serialized_string):
#     feature={
#       'image/height': tf.io.FixedLenFeature([], tf.int64),
#       'image/width': tf.io.FixedLenFeature([], tf.int64),
#       'image/filename': tf.io.FixedLenFeature([], tf.string),
#       'image/source_id': tf.io.FixedLenFeature([], tf.string),
#       'image/encoded': tf.io.FixedLenFeature([], tf.string),
#       'image/format': tf.io.FixedLenFeature([], tf.string),
#       'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
#       'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
#       'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
#       'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
#       'image/object/class/text': tf.io.VarLenFeature(tf.string),
#       'image/object/class/label': tf.io.VarLenFeature(tf.int64),
#     }

    
#     parsed_record = tf.io.parse_single_example(serialized_string, feature)
#     image = tf.io.decode_jpeg(parsed_record['image/encoded'])
#     xmins = tf.sparse.to_dense(parsed_record['image/object/bbox/xmin']).numpy()
#     xmaxs = tf.sparse.to_dense(parsed_record['image/object/bbox/xmax']).numpy()
#     ymins = tf.sparse.to_dense(parsed_record['image/object/bbox/ymin']).numpy()
#     ymaxs = tf.sparse.to_dense(parsed_record['image/object/bbox/ymax']).numpy()
    
#     bb = [[ymins[i], xmins[i], ymaxs[i], xmaxs[i]] for i in range(len(xmins))]

#     return image, bb

In [ ]:
# train_set = tf.data.TFRecordDataset(os.path.join(folders["DATA_PATH"],"train.tfrec"))

In [ ]:
# ds = train_set.take(1)
# for sample in ds:
#     image, bb = deserialize_example(sample)
    
# boxes = np.asarray(bb)
# num_boxes = len(boxes)

# img = plot_detections(np.array(image), boxes, np.ones(num_boxes, dtype='int32'), np.ones(num_boxes), category_index)
# plt.figure(figsize=(20,10))
# plt.imshow(img)
# plt.show()

In [ ]:
# del train_set

# Config Model

In [ ]:
!cp {os.path.join(folders['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(folders['CHECKPOINT_PATH'])}

In [ ]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
config

In [ ]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

In [ ]:
pipeline_config.model.ssd.num_classes = len(labels)
pipeline_config.model.ssd.image_resizer.keep_aspect_ratio_resizer.min_dimension = 1280
pipeline_config.model.ssd.image_resizer.keep_aspect_ratio_resizer.max_dimension = 1280
pipeline_config.train_config.data_augmentation_options[1].random_scale_crop_and_pad_to_square.output_size = 1280
pipeline_config.model.ssd.box_coder.faster_rcnn_box_coder.y_scale = 10.0
pipeline_config.model.ssd.box_coder.faster_rcnn_box_coder.x_scale = 10.0
pipeline_config.model.ssd.box_coder.faster_rcnn_box_coder.height_scale = 5.0
pipeline_config.model.ssd.box_coder.faster_rcnn_box_coder.width_scale = 5.0
pipeline_config.train_config.batch_size = 2
# pipeline_config.train_config.fine_tune_checkpoint = os.path.join(folders['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint = "../input/tfodtoutput/export7-D2-Plus10_000/export/checkpoint/ckpt-0"
pipeline_config.train_config.use_bfloat16 = False
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_config.sync_replicas = False
pipeline_config.train_config.replicas_to_aggregate = 1
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = 1e-3
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = 1e-4
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.total_steps = 10000
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_steps = 2000
pipeline_config.model.ssd.post_processing.batch_non_max_suppression.score_threshold = 1e-8
pipeline_config.model.ssd.post_processing.batch_non_max_suppression.iou_threshold = 0.5
pipeline_config.train_config.num_steps = 10000
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(folders['DATA_PATH'], 'train.tfrec')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(folders['DATA_PATH'], 'valid.tfrec')]

In [ ]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

In [ ]:
!cat {files["PIPELINE_CONFIG"]}

In [ ]:
gc.collect()

# Train Model

In [ ]:
!python {files['TRAINING_SCRIPT']}\
    --model_dir={folders['CHECKPOINT_PATH']}\
    --pipeline_config_path={files['PIPELINE_CONFIG']}\
    --num_train_steps=10000

# Evaluate Model

In [ ]:
!python {files['TRAINING_SCRIPT']}\
    --model_dir={folders['CHECKPOINT_PATH']}\
    --pipeline_config_path={files['PIPELINE_CONFIG']}\
    --checkpoint_dir={folders['CHECKPOINT_PATH']}\
    --eval_timeout=0 

# Export Model

In [ ]:
!python {files['EXPORTER_SCRIPT']}\
    --input_type image_tensor \
    --pipeline_config_path={files['PIPELINE_CONFIG']} \
    --trained_checkpoint_dir={folders['CHECKPOINT_PATH']} \
    --output_directory={folders['OUTPUT_PATH']}

# Load Model from Checkpoints

In [ ]:
!ls {folders["CHECKPOINT_PATH"]}/ckpt-*.index

In [ ]:
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

ckps = glob.glob(folders["CHECKPOINT_PATH"]+"/ckpt-*.index")
ckps.sort(key=os.path.getmtime)

ckp_file = ckps[-1][:-6]
print(ckp_file)
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(ckp_file).expect_partial()

def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

# Prediction

In [ ]:
def get_image_pred(df, idx):

    image_np = load_image_into_np(df.iloc[idx].image_path)
    
    height, width, _ = image_np.shape
    input_tensor = tf.cast(np.expand_dims(image_np, 0), tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = detections['num_detections'][0].numpy().astype(np.int32) 
    bboxes = []
    scores = []
    classes = []
    DETECTION_THRESHOLD = 0.15
    
    for i in range(num_detections):
        score = detections['detection_scores'][0][i].numpy()
        
        if score < DETECTION_THRESHOLD:
            continue

        bboxes.append(list(detections['detection_boxes'][0][i].numpy()))
        scores.append(score)
        classes.append(1)
    img = plot_detections(image_np, np.array(bboxes), np.array(classes), np.array(scores), category_index, unc=True)
    return img

cnt = 10
fig, ax = plt.subplots(cnt, 2, figsize = (20,40))
for row in range(cnt):
    idx = np.random.randint(0,val_data_df.shape[0])
    gt = get_image_with_annotation(val_data_df, idx)
    pred = get_image_pred(val_data_df, idx)
    
    ax[row][0].imshow(gt)
    ax[row][0].set_xticks([])
    ax[row][0].set_yticks([])
    ax[row][0].set_title(f"GT_{idx}")
    
    ax[row][1].imshow(pred)
    ax[row][1].set_xticks([])
    ax[row][1].set_yticks([])
    ax[row][1].set_title(f"PR_{idx}")
plt.tight_layout()
plt.show()

In [ ]:
!zip ./my_model.zip -r {folders["OUTPUT_PATH"]}

<a href="./my_models.zip"> Download File my_model.zip </a>

# Submission

In [ ]:
env = greatbarrierreef.make_env()  
iter_test = env.iter_test() 

In [ ]:
DETECTION_THRESHOLD = 0.15

for (image_np, df) in iter_test:
    height, width, _ = image_np.shape
    
    input_tensor = tf.cast(np.expand_dims(image_np, 0), tf.float32)
    detections = detect_fn(input_tensor)
    
    num_detections = detections['num_detections'][0].numpy().astype(np.int32)
    predictions = []
    
    for index in range(num_detections):
        score = detections['detection_scores'][0][index].numpy()

        if score < DETECTION_THRESHOLD:
            continue

        bbox = detections['detection_boxes'][0][index].numpy()
        y_min = int(bbox[0] * height)
        x_min = int(bbox[1] * width)
        y_max = int(bbox[2] * height)
        x_max = int(bbox[3] * width)

        bbox_width = x_max - x_min
        bbox_height = y_max - y_min

        predictions.append(f'{score:.2f} {x_min} {y_min} {bbox_width} {bbox_height}')
        
        
    prediction_str = ' '.join(predictions)
    df['annotations'] = prediction_str
    env.predict(df)

sub_df = pd.read_csv('submission.csv')
sub_df.head()

In [ ]:
!rm -rf ./api
!rm -rf ./data
!rm -rf ./pre-trained-models